<a href="https://colab.research.google.com/github/cseebec/funding_neural_network/blob/main/AlphabetSoupCharity_Optimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (1).csv


In [3]:
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))

In [4]:
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis=1)
application_df = application_df.drop('NAME', axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Create a list of application types to be replaced
application_types_to_replace = list(application_count[application_count < 200].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C2570        1
C1732        1
C1580        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [12]:
# Split preprocessed data into our features and target arrays
X = application_df.drop(["IS_SUCCESSFUL"], axis=1)
y = application_df["IS_SUCCESSFUL"]

In [13]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model
Initial Attempt

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = len(X_train_scaled[0])
# There is no significance between 80 nodes in the first hidden layer and 30 nodes in the second hidden layer. 
# These numbers were randomly selected
layer1_nodes = 80
layer2_nodes = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1_nodes, activation="relu", input_dim=number_inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2_nodes, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# The activation functions for the hidden layers are subject to change. 
# However, the activation function for the output layer must be sigmoid because the output is binary and sigmoid is the only binary classifier

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Create a callback that saves the model's weights every 5 epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint = "checkpoints/weighs.{epoch}.hdf5"

callback = ModelCheckpoint(filepath = checkpoint,
                           verbose = 1,
                           save_weights_only = True,
                            save_freq="epoch",
                           period=5)

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5736 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7327
Epoch 5/100
802/804 [============================>.] - ETA: 0s - loss: 0.5506 - accuracy: 0.7310
Epoch 00005: saving model to checkpoints/weighs.5.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5488 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================]

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5613 - accuracy: 0.7268 - 497ms/epoch - 2ms/step
Loss: 0.5613264441490173, Accuracy: 0.7267638444900513


In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

I want an accuracy of at least 0.75 so I am going to make some changes to the neural network to try to improve the model

### Optimize The Model Attempt 1
Add another hidden layer

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = len(X_train_scaled[0])
layer1_nodes = 80
layer2_nodes = 30
layer3_nodes = 10

nn_model1 = tf.keras.models.Sequential()

# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=layer1_nodes, activation="relu", input_dim=number_inputs))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=layer2_nodes, activation="relu"))

# Third hidden layer
nn_model1.add(tf.keras.layers.Dense(units=layer3_nodes, activation="relu"))


# Output layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# The activation functions for the hidden layers are subject to change. 
# However, the activation function for the output layer must be sigmoid because the output is binary and sigmoid is the only binary classifier

# Check the structure of the model
nn_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5726 - accuracy: 0.7204
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5557 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7325
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7336
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5584 - accuracy: 0.7255 - 423ms/epoch - 2ms/step
Loss: 0.5584021210670471, Accuracy: 0.7254810333251953


### Attempt 1 Results
Adding another hidden layer to the neural network had such a small effect on the model's results that it is insignificant. That being said, having another hidden layer takes more computional power, so I will continue to use 2 hidden layers not 3

### Optimize The Model Attempt 2
Change one of the activation functions

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = len(X_train_scaled[0])
# There is no significance between 80 nodes in the first hidden layer and 30 nodes in the second hidden layer. 
# These numbers were randomly selected
layer1_nodes = 80
layer2_nodes = 30

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=layer1_nodes, activation="relu", input_dim=number_inputs))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=layer2_nodes, activation="tanh"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# The activation functions for the hidden layers are subject to change. 
# However, the activation function for the output layer must be sigmoid because the output is binary and sigmoid is the only binary classifier

# Check the structure of the model
nn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5698 - accuracy: 0.7219
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5513 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accura

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7271 - 466ms/epoch - 2ms/step
Loss: 0.5548086762428284, Accuracy: 0.7271137237548828


### Attempt 2 Results
Similar to the first attempt, changing the activation function from relu to tanh had such a small effect on the model that it is practically insignificant. However, even though the change was extremely small the model did perform better with tanh than relu so I am keep tanh as the activation function in my next attempt.

### Optimize The Model Attempt 3
Decrease the number of the nodes in both hidden layers

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = len(X_train_scaled[0])
layer1_nodes = 20
layer2_nodes = 10

nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=layer1_nodes, activation="relu", input_dim=number_inputs))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=layer2_nodes, activation="tanh"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# The activation functions for the hidden layers are subject to change. 
# However, the activation function for the output layer must be sigmoid because the output is binary and sigmoid is the only binary classifier

# Check the structure of the model
nn_model3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 20)                880       
                                                                 
 dense_14 (Dense)            (None, 10)                210       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5827 - accuracy: 0.7064
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7307
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7339
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7323
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5530 - accuracy: 0.7262 - 453ms/epoch - 2ms/step
Loss: 0.5530232191085815, Accuracy: 0.7261807322502136


### Attempt 3 Results
Similar to the first 2 attempts, changing the number of nodes in the hidden layers had such a small effect on the model that it is practically insignificant. 

### Optimize The Model Attempt 4
Decrease the number of bins

In [37]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (2).csv


In [38]:
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))

In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis=1)
application_df = application_df.drop('NAME', axis=1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [40]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [41]:
# Create a list of application types to be replaced
application_types_to_replace = list(application_count[application_count < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [42]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C2570        1
C1732        1
C1580        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [44]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 6000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
Other    10899
C2000     6074
Name: CLASSIFICATION, dtype: int64

In [45]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
# Split preprocessed data into our features and target arrays
X = application_df.drop(["IS_SUCCESSFUL"], axis=1)
y = application_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = len(X_train_scaled[0])
# There is no significance between 80 nodes in the first hidden layer and 30 nodes in the second hidden layer. 
# These numbers were randomly selected
layer1_nodes = 80
layer2_nodes = 30

nn_model4 = tf.keras.models.Sequential()

# First hidden layer
nn_model4.add(tf.keras.layers.Dense(units=layer1_nodes, activation="relu", input_dim=number_inputs))

# Second hidden layer
nn_model4.add(tf.keras.layers.Dense(units=layer2_nodes, activation="tanh"))

# Output layer
nn_model4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# The activation functions for the hidden layers are subject to change. 
# However, the activation function for the output layer must be sigmoid because the output is binary and sigmoid is the only binary classifier

# Check the structure of the model
nn_model4.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 80)                3040      
                                                                 
 dense_20 (Dense)            (None, 30)                2430      
                                                                 
 dense_21 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,501
Trainable params: 5,501
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Compile the model
nn_model4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [51]:
# Train the model
fit_model = nn_model4.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5776 - accuracy: 0.7147
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5651 - accuracy: 0.7265
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5633 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5614 - accuracy: 0.7275
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7285
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - accuracy: 0.7301
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5587 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5583 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5580 - accura

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5635 - accuracy: 0.7259 - 424ms/epoch - 2ms/step
Loss: 0.5634517073631287, Accuracy: 0.7259474992752075
